# BIC - CW

### Importing the libraries

In [5]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

# Import PySwarms as instructed by pyswarms webpage
import pyswarms as ps

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Implement a multi-layer ANN architecture

### Data Preprocessing

#### Importing the dataset (Churn Modelling)

In [6]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

#### Encoding gender label

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

#### Encoding geography label

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

### Building the ANN

#### Initialising the ANN and indicating number of neurons

In [10]:
n_inputs = 12
n_hidden = int(input("Enter number of neurons: "))
n_classes = 2

num_samples = len(y)

Enter number of neurons: 20


In [4]:
def logits_function(p):

    # Roll-back the weights and biases
    W1 = p[0:240].reshape((n_inputs,n_hidden))
    b1 = p[240:260].reshape((n_hidden,))
    W2 = p[260:300].reshape((n_hidden,n_classes))
    b2 = p[300:302].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1.astype(float))     # Activation in Layer 1
    logits = a1.dot(W2) + b2 # Pre-activation in Layer 2
    return logits          # Logits for Layer 2

In [7]:
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """

    logits = logits_function(params)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood

    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss


In [8]:
def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [9]:
%%time
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9, 'k':2, 'p':2}

# Call instance of PSO
dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
optimizer = ps.single.LocalBestPSO(n_particles=50, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=500)

2020-11-20 22:50:40,940 - pyswarms.single.local_best - INFO - Optimize for 500 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 2, 'p': 2}
pyswarms.single.local_best: 100%|██████████|500/500, best_cost=0.488
2020-11-20 22:52:23,305 - pyswarms.single.local_best - INFO - Optimization finished | best cost: 0.4875497012103313, best pos: [-1.05823383e-01 -1.80529829e-01  3.38186431e-01  5.91927827e-01
  7.48536514e-01  4.53552493e-01  6.04790155e-01  1.26560671e-01
  5.93218111e-01  1.33883306e-01  5.57001552e-01 -1.45787893e-01
 -2.83980850e-01  3.00044412e-01  2.94468776e-01  4.31856810e-01
  5.24801073e-01  3.34410708e-01 -2.58656603e-03  4.82361283e-01
  2.89888157e-01  1.47474110e-01 -8.82105480e-02  5.03887669e-01
 -2.41371093e-01  1.66089321e-01 -2.68149698e-02  3.01334112e-01
  2.62655387e-02  2.60981464e-01  2.66085111e-01  9.23893261e-02
  7.83989015e-01  1.78323587e-01 -1.13611825e-01  4.37231920e-01
  1.29991929e+00  8.20001347e-01 -7.53328957e-01  4.27420376e-01
 -5.13881974e-0

CPU times: user 1min 41s, sys: 838 ms, total: 1min 42s
Wall time: 1min 42s


In [10]:
def predict(pos):
    """
    Use the trained weights to perform class predictions.

    Inputs
    ------
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """
    logits = logits_function(pos)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [11]:
(predict(pos) == y).mean()

0.7984031936127745

### Number of Neurons and Layers Configuration

In [ ]:
neurons = int(input("Enter number of neurons: "))

In [ ]:
layers = []
n = int(input("Enter number of layers (1-5): "))
if n < 6 and n > 0:
    for i in range (0, n):
        print("Hyperbolic tangent layer must run every time.")
        print("Available activation functions: linear, sigmoid, gaussian, cosine, null")
        layer = input("Enter name of desired activation function: ")
    
        layers.append(layer)    
else:
    print("Wrong number of layers entered")

### Adding the input layer and the first hidden layer with linear AF

In [ ]:
if "linear" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))

### Adding the second hidden layer with Sigmoid AF

In [ ]:
if "sigmoid" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='sigmoid'))

### Adding the third hidden layer with Gaussian AF

In [ ]:
if "gaussian" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='gaussian'))

### Adding the fourth hidden layer with Cosine AF

In [ ]:
if "cosine" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='cosine'))

### Adding the fifth hidden layer with Null AF

In [ ]:
if "null" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='nullact'))

### Adding the output layer with Hyperbolic Tangent AF

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='tanh'))

## Part 3 - Training the ANN

### Compiling the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [ ]:
def forward_prop(params):
    
    for x_train, y_train in dataset:
        logits = ann(params)
    
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood

    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss

In [ ]:
def f(x):
    n_particles = x.shape[0]
    print(n_particles)
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [ ]:
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
dimensions = 54
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=1000)

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Solution**

In [ ]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

### Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)